In [1]:
import os,time
import rasterio #for reading images
import napari
import re
import multiprocessing as mlt
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd

from matplotlib import path

directory = os.path.join(os.getcwd(), "ee_data")

def runNapari(img):
    
    viewer = napari.view_image(img)
    napari.run()
    return viewer

def getMap(fileName):

    with rasterio.open('ee_data//Engilchek_2001-10-02.tif') as src:
        img = src.read() 
    napariConcurrent = mlt.Process(target=runNapari, args=img)
    viewer = napariConcurrent.get()
    input("Press ENTER after annotation")
    while True:
        try:
            path_dimention = getPolygonMasks(viewer)
            break
        except KeyError:
            print("Annotate on UI")
            time.sleep(5)

    paths = containsWithin(path_dimention, img)
    ice_contain_map = [containsWithin(path,img) for path in paths["ice"]]
    non_ice_contain_map = [containsWithin(path,img) for path in paths["non_ice"]]
    return ice_contain_map,non_ice_contain_map
    

def getPolygonMasks(viewer)-> list:

    ice_layers = {str.lower(layer.name):layer.data for layer in viewer.layers if re.match("^ice.*", str.lower(layer.name))}
    non_ice_layers= {str.lower(layer.name):layer.data for layer in viewer.layers if re.match("^not_ice.*", str.lower(layer.name))}
    ice_coordinate_list, non_ice_coordinate_list = list(),list()

    for i, (x,y) in enumerate(ice_layers.items()):
        ice_coordinate_list.append(np.delete(y[0],(0),axis=1))
    for i, (x,y) in enumerate(non_ice_layers.items()):
        non_ice_coordinate_list.append(np.delete(y[0],(0),axis=1))

    path_results = {"ice":ice_coordinate_list, "non_ice":non_ice_coordinate_list}
    return path_results

    
def containsWithin(path_dimention, img):

    glacier = path.Path(path_dimention)
    indices = np.where(np.all(img == img, axis=0))
    pixels = np.array(list(zip(indices[0],indices[1])))
#     return pixels
    return glacier.contains_points(pixels).reshape(img.shape[1:])


def getPixelMask(img,paths):

    iceMask = np.where(np.logical_or.reduce([containsWithin(path,img) for path in paths["ice"]]))
    nonIceMask = np.where(np.logical_or.reduce([containsWithin(path,img) for path in paths["non_ice"]]))

    ice_coordinates = list(zip(iceMask[0],iceMask[1]))
    non_ice_coordinates = list(zip(nonIceMask[0],nonIceMask[1]))    
    iceData,nonIceData=list(),list()
    for (x,y) in ice_coordinates:
        iceData.append(np.rollaxis(img,0,3)[x,y,:])
    for (x,y) in non_ice_coordinates:
        nonIceData.append(np.rollaxis(img,0,3)[x,y,:])
    return np.array(iceData), np.array(nonIceData)


In [13]:
pth = r"C:\Users\marke\Documents\DSC180A-Q1\ee_data\Engilchek_2000-10-31.tif"
with rasterio.open(pth) as src:
    first_band = src.read(1)
    img = src.read()
    
# test_viewer = runNapari(img)


In [4]:
getPolygonMasks(test_viewer)

{'ice': [array([[ 72.86350764,  75.76788799],
         [ 68.23940283,  90.12695029],
         [ 68.23940283,  98.40166415],
         [ 65.80566346, 114.46434401],
         [ 64.83216771, 134.17763292],
         [ 64.83216771, 134.17763292],
         [ 63.85867196, 150.24031278],
         [ 63.85867196, 150.24031278],
         [ 77.73098638, 157.54153089],
         [ 79.19123001, 147.07645159],
         [ 79.19123001, 147.07645159],
         [ 80.89484757, 114.22097007],
         [ 82.35509119,  94.7510551 ],
         [ 82.35509119,  94.7510551 ],
         [ 82.84183906,  75.52451406],
         [ 71.40326401,  75.76788799]]),
  array([[ 63.37192409, 160.94876601],
         [ 59.96468897, 178.71506343],
         [ 58.99119322, 187.47652517],
         [ 57.04420172, 198.18497841],
         [ 57.04420172, 204.0259529 ],
         [ 60.93818472, 205.72957046],
         [ 60.93818472, 205.72957046],
         [ 68.48277677, 198.18497841],
         [ 68.48277677, 198.18497841],
         [ 71.89

In [14]:
testdata = getPixelMask(img, getPolygonMasks(test_viewer))

IndexError: tuple index out of range

In [8]:
testdata

(array([[0.27219427],
        [0.4373811 ],
        [0.37747154],
        ...,
        [1.2533133 ],
        [1.4252207 ],
        [1.3195981 ]], dtype=float32),
 array([[ 0.5352343 ],
        [ 0.64155304],
        [ 0.60059613],
        [ 0.40949854],
        [ 0.43538994],
        [ 0.6365392 ],
        [ 0.890106  ],
        [ 0.6989406 ],
        [ 0.6683228 ],
        [ 0.6087554 ],
        [ 0.63237643],
        [ 0.6825612 ],
        [ 0.71229875],
        [ 0.6893015 ],
        [ 0.69073015],
        [ 0.7050067 ],
        [ 0.6822219 ],
        [ 0.48999244],
        [ 0.49375206],
        [ 0.5894401 ],
        [ 0.5075021 ],
        [ 0.42162853],
        [ 0.5352582 ],
        [ 0.4387196 ],
        [ 0.5876501 ],
        [ 0.91215193],
        [ 0.7003055 ],
        [ 0.70070505],
        [ 0.63574904],
        [ 0.64642084],
        [ 0.6245105 ],
        [ 0.83196354],
        [ 0.9814379 ],
        [ 0.8540297 ],
        [ 0.67955166],
        [ 0.6823704 ],
        [ 

In [7]:
np.array(["Ice"]*testdata[0].shape[0])

array(['Ice', 'Ice', 'Ice', ..., 'Ice', 'Ice', 'Ice'], dtype='<U3')